In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    """
    This class encapsulates the variables associated with Model Training
    """

    root_directory: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_augmentation_required: bool
    params_image_size: list
    params_validation_split: float
    params_interpolation: str
    params_rotation_range: int
    params_horizontal_flip: bool
    params_width_shift_range: float
    params_height_shift_range: float
    params_shear_range: float
    params_zoom_range: float
    params_shuffle: bool
    params_validation_subset: str
    params_training_subset: str

In [ ]:
import os
from pathlib import Path
from src.cnn_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnn_classifier.utils.common import read_yaml, create_directories
from src.cnn_classifier.entity.config_entity import ModelTrainingConfig

In [ ]:
"""
This is the configuration manager for all the stages of pipelines for this Projects.
It will have all the configuration methods encapsulated in a class
"""

import os
from pathlib import Path
from src.cnn_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnn_classifier.utils.common import read_yaml, create_directories
from src.cnn_classifier.entity.config_entity import (
    DataIngestionConfig,
    PrepareModelArchitectureConfig,
    ModelTrainingConfig,
    ModelEvaluationConfig,
    PredictionPipelineConfig,
)


class ConfigurationManager:
    """
    This is the configuration manager for all the stages of pipelines for this Projects
    """

    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories(directory_list=[self.config.artifacts_root])

 
    def get_model_training_config(self) -> ModelTrainingConfig:
        """
        This method is to fetch the config details for Model Training
        """
        config = self.config.model_training
        prepare_model_config = self.config.prepare_model
        training_data = self.config.data_ingestion.unzip_directory

        create_directories([Path(config.root_directory)])

        model_training_config = ModelTrainingConfig(
            root_directory=Path(config.root_directory),
            trained_model_path=Path(config.trained_model_path),
            updated_model_path=Path(prepare_model_config.updated_model_path),
            training_data=Path(training_data),
            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE,
            params_augmentation_required=self.params.AUGMENTATION,
            params_image_size=self.params.IMAGE_SIZE,
            params_validation_split=self.params.VALIDATION_SPLIT,
            params_interpolation=self.params.INTERPOLATION,
            params_rotation_range=self.params.ROTATION_RANGE,
            params_horizontal_flip=self.params.HORIZONTAL_FLIP,
            params_width_shift_range=self.params.WIDTH_SHIFT_RANGE,
            params_height_shift_range=self.params.HEIGHT_SHIFT_RANGE,
            params_shear_range=self.params.SHEAR_RANGE,
            params_zoom_range=self.params.ZOOM_RANGE,
            params_shuffle=self.params.SHUFFLE,
            params_validation_subset=self.params.VALIDATION_SUBSET,
            params_training_subset=self.params.TRAINING_SUBSET,
        )

        return model_training_config


In [ ]:
from pathlib import Path
import tensorflow as tf
from src.cnn_classifier.entity.config_entity import ModelTrainingConfig

In [ ]:
class ModelTraining:
    """
    This class encapsulates methods that are used for
    data preprocessing, train test split and model training
    """

    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        self.model: tf.keras.models = None
        self.validation_data_generator: tf.data.Dataset = None
        self.training_data_generator: tf.data.Dataset = None
        self.steps_per_epoch: int = 0
        self.validation_steps: int = 0

    def get_pretrained_model(self):
        """
        Loads pretrained model from artifacts
        """
        self.model = tf.keras.models.load_model(self.config.updated_model_path)

    def image_preprocessing(self):
        """
        Performs Data Augmentation and Preprocess the image data using ImageDataGenerator
        """
        data_generator_kwargs = dict(
            rescale=1.0 / 255, validation_split=self.config.params_validation_split
        )

        data_flow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation=self.config.params_interpolation,
        )

        validation_image_datagenerator = (
            tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_kwargs)
        )

        self.validation_data_generator = (
            validation_image_datagenerator.flow_from_directory(
                directory=self.config.training_data,
                subset=self.config.params_validation_subset,
                shuffle=self.config.params_shuffle,
                **data_flow_kwargs,
            )
        )

        if self.config.params_augmentation_required:
            training_image_datagenerator = (
                tf.keras.preprocessing.image.ImageDataGenerator(
                    rotation_range=self.config.params_rotation_range,
                    horizontal_flip=self.config.params_horizontal_flip,
                    width_shift_range=self.config.params_width_shift_range,
                    height_shift_range=self.config.params_height_shift_range,
                    shear_range=self.config.params_shear_range,
                    zoom_range=self.config.params_zoom_range,
                    **data_generator_kwargs,
                )
            )
        else:
            training_image_datagenerator = validation_image_datagenerator

        self.training_data_generator = training_image_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset=self.config.params_training_subset,
            shuffle=self.config.params_shuffle,
            **data_flow_kwargs,
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Method to save the model definition"""
        model.save(path)

    def train(self):
        """Training the prepared model"""
        self.steps_per_epoch = (
            self.training_data_generator.samples
            // self.training_data_generator.batch_size
        )
        self.validation_steps = (
            self.validation_data_generator.samples
            // self.validation_data_generator.batch_size
        )

        self.model.fit(
            self.training_data_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.validation_data_generator,
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)


In [ ]:
from src.cnn_classifier.components.model_trainer import ModelTraining
from src.cnn_classifier.config.configuration import ConfigurationManager
from src.cnn_classifier.logger import logging

In [ ]:
STAGE_NAME = "Model Training Pipeline"

class ModelTrainingPipeline:
    """This class enapsulates the methods to trigger the Model Training Pipeline"""

    def __init__(self):
        pass

    def main(self):
        """Main method to call methods for model training"""
        logging.info(
            "Inside src.cnn_classifier.components.pipeline.ModelTrainingPipeline"
        )
        config = ConfigurationManager()
        model_training_config = config.get_model_training_config()
        logging.info(
            "Completed execution get_model_training_config method of \
                src.cnn_classifier.config.configuration.ConfigurationManager"
        )
        model_training = ModelTraining(config=model_training_config)

        model_training.get_pretrained_model()
        logging.info(
            "Completed execution get_pretrained_model method of \
                src.cnn_classifier.components.model_trainer.ModelTraining"
        )
        model_training.image_preprocessing()
        logging.info(
            "Completed execution image_preprocessing method of \
                src.cnn_classifier.components.model_trainer.ModelTraining"
        )
        model_training.train()
        logging.info(
            "Completed Model Training using train method of \
                src.cnn_classifier.components.model_trainer.ModelTraining"
        )
        logging.info(
            "Completed execution of \
                src.cnn_classifier.components.model_trainer.ModelTraining"
        )

In [ ]:
try:
    logging.info(">>>>>> Started %s <<<<<<", STAGE_NAME)
    obj = ModelTrainingPipeline()
    obj.main()
    logging.info(">>>>>> %s completed <<<<<<\n\n", STAGE_NAME)
except Exception as error:
    logging.exception(error)
    raise error